# Prepare environment

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# clone repo via ssh

!cp /content/drive/MyDrive/.ssh /root/  -r
!chmod 644 /root/.ssh/known_hosts
!chmod 600 /root/.ssh/id_rsa
!ssh -T git@github.com

!git clone -q --recurse-submodules git@github.com:annapanfil/punctuation_prediction.git

In [ ]:
%cd punctuation_prediction/
!pip install -q -r requirements.txt

In [ ]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = "annapanfil"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "f3ab7a17321dd66221aae96b7dbc9f43434a812d"

# Conduct experiments

In [ ]:
!python3 src/dataset_creation.py --newlines=false

## Done

In [ ]:
# models
sh = """
#!/bin/bash

# Function to run training
run_training() {
    pretrained_model=$1
    language=$2
    epochs=$3
    command="python src/train.py --pretrained-model=${pretrained_model} --language=${language} --epoch=${epochs} --cuda=True"
    echo "Running command: $command"
    $command
}

# List of different models to try
models=("herbert-base" "bert-multiling-uncased" "roberta-multiling" "polish-roberta")
language="polish"
epochs=10

# Run training for each model
for model in "${models[@]}"
do
    run_training "$model" "$language" "$epochs"
done
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

In [ ]:
# !python src/train.py --cuda=True --pretrained-model=bert-base-uncased --freeze-bert=False --lstm-dim=-1 --language=english --seed=1 --lr=5e-6 --epoch=8 --use-crf=False --augment-type=all  --augment-rate=0.15 --alpha-sub=0.4 --alpha-del=0.4 --data-path=data --save-path=out
# !python src/train.py --pretrained-model=herbert-base --language=polish --epoch=10

In [ ]:
# !python src/inference.py --pretrained-model=bert-base-uncased --weight-path=out/weights.pt --language=en --in-file=data/test_en.txt --out-file=data/test_en_out.txt
# !python src/inference.py --pretrained-model=herbert-base --weight-path=out/weights.pt --language=pl --in-file=data/test_pl.txt --out-file=data/test_pl_out.txt

In [ ]:
# LSTM and freezing BERT

!python src/train.py --freeze-bert=False --lstm=bi --name="Bi, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"
!python src/train.py --freeze-bert=True --lstm=bi --name="Bi, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=20 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"

!python src/train.py --freeze-bert=False --lstm=uni --name="Uni, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"
!python src/train.py --freeze-bert=True --lstm=uni --name="Uni, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=20 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"

!python src/train.py --freeze-bert=False --lstm=none --name="No LSTM, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"
!python src/train.py --freeze-bert=True --lstm=none --name="No LSTM, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=20 --log=true --data-variation=allpunct --experiment-name="LSTM and freezing BERT"

## TODO

In [3]:
# hyperparameters

sh = """
#!/bin/bash

# Function to run training
run_training() {
    lr=$1
    name="lr $lr"
    command="python src/train.py --name=${name} --lr=${lr} --epoch=10 --freeze-bert=False --lstm=none --language=polish --cuda=True --pretrained-model=polish-roberta --log=true --data-variation=allpunct --experiment-name=hyperparameters"
    echo "Running command: $command"
    $command
}

# List of different models to try
lrs=(1e-1 1e-2 1e-3 1e-4 1e-5)

# Run training for each model
for lr in "${lrs[@]}"
do
    run_training "$lr"
done
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

Running command: python src/train.py --name=lr 1e-1 --lr=1e-1 --epoch=10 --freeze-bert=False --lstm=none --language=polish --cuda=True --pretrained-model=polish-roberta --log=true --data-variation=allpunct --experiment-name=hyperparameters


/mnt/dane/.conda/envs/mgr/lib/python3.12/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/mnt/dane/.conda/envs/mgr/lib/python3.12/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v4.40. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
usage: train.py [-h] [--name NAME] [--experiment-name EXPERIMENT_NAME]
                [--data-path DATA_PATH] [--data-variation DATA_VARIATION]
                [--language LANGUAGE] [--cuda CUDA] [--seed SEED]
                [--pretrained-model PRETRAINED_MODEL]
                [--freeze-bert FREEZE_BERT] [--lstm LSTM]
                [--lstm-dim LSTM_DIM] [--use-crf USE_CRF]
                [--sequence-le